In [1]:
import numpy as np
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
import statsmodels.api as sm
from scipy.stats import zscore
import scipy as sp
from numpy import random, linalg
from scipy import sparse, stats
import itertools as it
from sklearn.preprocessing import StandardScaler as scaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from joblib import Parallel, delayed
import multiprocessing


matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.preamble'] = [
    r'\usepackage{amssymb}',
    r'\usepackage{amsmath}',
    r'\usepackage{xcolor}',
    r'\renewcommand*\familydefault{\sfdefault}']
matplotlib.rcParams['pgf.texsystem'] = 'pdflatex'
matplotlib.rcParams['pgf.preamble'] = [
    r'\usepackage[utf8x]{inputenc}',
    r'\usepackage{amssymb}',
    r'\usepackage[T1]{fontenc}',
    r'\usepackage{amsmath}',
    r'\usepackage{sansmath}']

inv, ax, norm = np.linalg.inv, np.newaxis, np.linalg.norm
randint = np.random.randint


C:\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Question 1

#### a)

In [2]:

def lasso_objective(b, y, X, lmbda):

    # Question 1 Part A

    """
        Function that accepts the guess for the parameter vector and LASSO penalty multipler λ,
         and computes the LASSO objective function based on the input data.
        :param b: Parameter vector.
        :param y: Outcome variable, vector of size N.
        :param X: Covariate variables (may or may not include ι), matrix of size N x P.
        :param lmbda: LASSO penalty.
        :return: Objective function evaluated using inputs.
    """

    # Return the objective function if matrix multiplication Xβ is compatible.
    try:
        obj = np.square(y - X @ b).sum()/(2*y.size) + lmbda * norm(b, ord=1)
        return obj
    except:
        print("Error: The number of covariates is not compatible with given coefficient vector.")
        return np.inf


def dual_sol(bj, lmbda):

    """
        Function that returns the solution for a single coordinate in the Cyclic
         Coordinate Descent algorithm given the OLS coordinate estimate and the
         LASSO penalty multipler.
        :param bj: OLS estimate for coordinate j.
        :param lmbda: LASSO penalty multiplier.
        :return: LASSO coordinate estimate.
    """

    if bj < - lmbda:
        return bj + lmbda
    elif bj > lmbda:
        return bj - lmbda
    else:
        return 0


def lambda_zero(y, X, standardized=False):

    # Question 1 Part D

    """
        Function that computes the smallest penalty at which the LASSO estimate is exactly equal to zero.
        :param y: Outcome variable, vector of size N.
        :param X: Covariate variables (may or may not include ι), matrix of size N x P.
        :param standardized: Indicator for whether the data has been standardized.
        :return: The lambda penalty.
    """

    # Detect if a constant term is included
    iota = (X[:, 0] == X[:, 0].mean()).all()

    # Trim out constant term
    if iota:
        X = X[:, 1:]

    if standardized is False:
        X, y = zscore(X, axis=0), zscore(y)
    lmbda_max = np.max(np.abs(y.T @ X) / y.size)

    return lmbda_max


def lasso_cdg(b_start, y, X, lmbda, eps=1e-7, max_iter=5000, standardized=False,
              active_set=False, active_set_cycle=10, safe=False):

    # Question 1 Part B + C + E

    """
        Function that performs the LASSO estimation through the Cyclic Coordinate Descent algorithm. Additional
         options for using the Active Set Strategy and the SAFE algorithm are provided.

        :param b_start: Initial guess for the parameter vector (may or may not include b0, which will be 
         trimmed out if so)
        :param y: Outcome variable, vector of size N.
        :param X: Covariate variables (may or may not include ι), matrix of size N x P.
        :param lmbda: LASSO penalty multiplier.
        :param eps: Norm stopping criterion.
        :param max_iter: Iteration number stopping criterion.
        :param standardized: Indicator for whether the data has been standardized.

        :param active_set: Option for using the Active Set strategy to increase the speed of the algorithm.
        :param active_set_cycle: The frequency at which all covariates are updated in CDG rather than those in
         the Active Set. This option is not used if "active_set" is False.
        :param safe: Option for using the SAFE strategy to discard covariates.

        :return: List containing:
            - :estimate: final coefficient vector estimate,
            - :objectives: vector containing LASSO objective function values
            - :steps: vector containing norm of difference in estimated parameter vectors
            - :status: string regarding which stopping criterion was used.
    """

    p, N, b_guess = b_start.size, y.size, b_start
    y_mean, y_std = y.mean(), y.std()

    if N != X.shape[0]:
        print("Error: Covariate matrix is incompatible with outcome variable.")
        return None
    elif p != X.shape[1]:
        print("Error: Covariate matrix is incompatible with parameter vector.")
        return None

    # Detect if a constant term is included
    iota = (X[:, 0] == X[:, 0].mean()).all()

    # Trim out constant term
    if iota:
        X, b_guess = X[:, 1:], b_guess[1:]
        p = p - 1

    # Computing sample mean and std. dev. for the pre-standardized data.
    X_mean, X_std = X.mean(axis=0), X.std(axis=0)

    # Standardize data if not done so
    if standardized is False:
        X, y = zscore(X, axis=0), zscore(y)
        # X, y = (X - X.mean(axis=0)[ax, :])/X.std(axis=0)[ax, :], y - y.mean()

    # LASSO objective
    lasso_obj = lambda b: lasso_objective(b, y, X, lmbda)

    # Default options for Active Set and SAFE strategies
    active_range = lambda x: np.arange(0, p)
    safe_range = np.arange(0, p)

    # Implementing Active Set Strategy
    if active_set:
        active_range = lambda x: np.where(x != 0)[0]

    # Implementing SAFE Strategy
    if safe:

        # Find lambda_max: minimum value of lambda such that LASSO estimate is zero
        lmbda_max = lambda_zero(y=y, X=X, standardized=True)

        # Assign Boolean condition for covariates that can be discarded using the SAFE condition
        safe_condition = (X.T @ y).squeeze() < lmbda - norm(X, axis=0) * norm(y) * (lmbda_max - lmbda)/lmbda

        # Find indices corresponding to "relevant" covariates
        safe_range = np.where(~safe_condition)[0]

        # Set discarded covariates to have zero estimate under LASSO
        b_guess[safe_condition] = 0

    # Empty dictionary to store results of LASSO estimation for a given value of lambda.
    key_dict = {"estimate", "objectives", "steps", "status"}
    output = dict([(key, []) for key in key_dict])

    niter, dist = 0, 1000

    # While loop to perform LASSO minimization using two stopping criterion.
    while niter < max_iter and dist > eps:

        b_update = np.zeros(shape=b_guess.shape)

        # Active set strategy
        if niter % active_set_cycle == 0:

            # Using SAFE range for every active_set_cycle number of iterations.
            range_j = safe_range

        else:

            # Apply active-set range.
            range_j = active_range(b_guess)

        for j in range_j:

            # Extract j^{th} covariate vector
            Xj = X[:, j].reshape(-1, 1)

            # Compute OLS solution for β_j taking β_{-j} as given
            bj = b_guess[j] + Xj.T @ (y - X @ b_guess)/N

            # Update guess for j^{th} coordinate using LASSO closed form solution under CDG.
            b_update[j] = dual_sol(bj, lmbda)

        # Compute the distance between the successive LASSO estimates.
        b0 = y_mean - np.dot(X_mean, b_update)
        dist = norm(b_update - b_guess, ord=np.inf)

        output["estimate"].append(np.append(b0, b_update))
        output["objectives"].append(lasso_obj(b_update))
        output["steps"].append(dist)

        niter = niter + 1

        # Update guess for LASSO estimate in the next cycle of CDG.
        b_guess = b_update

        if dist <= eps:
            output["status"] = "convergence"

    if niter >= max_iter:
        output["status"] = "max_iter exceed"

    return output


def lasso_wrapper_sequential(b_start, y, X, standardized=False, num_lambda=100, min_factor=0.001,
                             warm_start=False, k_fold=False, num_k=5):

    # Question 1 Part F (a) + (b) + G

    """
        Function that performs the LASSO estimation through CDG + Active Set + SAFE for a pre-determined
         sequence of λ. Includes option to use the warm start feature for the initial guess of the LASSO
         estimate in successive iterations over λ values, and for K-fold cross-validation.

        :param b_start: Initial guess for the parameter vector (may or may not include b0, which will be
         trimmed out if so)
        :param y: Outcome variable, vector of size N.
        :param X: Covariate variables (may or may not include ι), matrix of size N x P.
        :param standardized: Indicator for whether the data has been standardized.

        :param num_lambda: Number of λ values to include in the sequence.
        :param min_factor: The minimum value of λ as a fraction of λ_max

        :param warm_start: Option for using warm start initial conditions.

        As such the lambda sequence is given by:
        Sequence(λ) = log([λ_max * min_factor : num_lambda : λ_max])

        :param k_fold: Option to compute cross-validation results of lambda in a K-fold CV algorithm.
        :param num_k: The number of K-folds to use. Used only when k_fold is True.

        :return: List containing:
            - :estimate: Final coefficient vector estimates across different λ values,
            - :objectives: Vector containing LASSO objective across different λ values,
            - :lambda: string regarding which stopping criterion was used.
    """

    # Computing maximum lambda before LASSO estimate is exactly zero.
    lmbda_max = lambda_zero(y, X, standardized=standardized)

    print('max = {}'.format(lmbda_max))

    # Computing sequence of lambdas over which LASSO is run based on the specification.
    lmbda_vec = np.linspace(start=min_factor, stop=1, num=num_lambda) * lmbda_max

    if k_fold is False:  # Running procedure with no K-fold cross-validation.

        # Empty dictionary to store results for each value of lambda.
        key_dict = {"lambda", "estimate", "objective", "status"}
        output = dict([(key, []) for key in key_dict])

        # Looping over the lambda vector.
        for il, lmbda in enumerate(np.flipud(lmbda_vec)):

            # Running lasso_cdg for the given value of lambda. We use both the active_set and SAFE strategies.
            lasso_est = lasso_cdg(b_start, y, X, lmbda, eps=1e-6, max_iter=1000, standardized=standardized,
                                  active_set=True, active_set_cycle=10, safe=True)

            output["lambda"].append(lmbda)
            output["estimate"].append(lasso_est['estimate'][-1])
            output["objective"].append(lasso_est['objectives'][-1])
            output["status"].append(lasso_est['status'])

            if warm_start:

                b_start = lasso_est['estimate'][-1][:, ax]

    else:  # Running procedure with K-fold cross-validation of lambda.

        # Empty dictionary to store CV error for each value of lambda.
        key_dict = {"lambda", "CVError"}
        output = dict([(key, []) for key in key_dict])

        kf = KFold(n_splits=num_k)
        iota = (X[:, 0] == X[:, 0].mean()).all()
        b_start_k = [b_start] * num_k

        # Looping over the lambda vector.

        for il, lmbda in enumerate(np.flipud(lmbda_vec)):

            split_num, cv_error_k = 0, np.zeros(num_k)

            # Looping over each K-fold

            for train_index, test_index in kf.split(X):

                # Training to obtain LASSO estimate for the given lambda

                X_train, y_train = X[train_index, :], y[train_index]

                lasso_res_k = lasso_cdg(b_start=b_start_k[split_num], y=y_train, X=X_train, lmbda=lmbda,
                                        standardized=standardized,
                                        active_set=True, active_set_cycle=10, safe=True)

                lasso_est_k = lasso_res_k['estimate'][-1]

                if warm_start:

                    b_start_k[split_num] = lasso_est_k[:, ax]

                # Using the test data to construct the prediction error

                X_test, y_test = X[test_index, :], y[test_index]

                cv_error_k[split_num] = np.mean(np.square(y_test - X_test @ lasso_est_k[~iota * 1:]))

                split_num = split_num + 1

            # Storing the lambda value and the CV error

            output["lambda"].append(lmbda)
            output["CVError"].append(cv_error_k.sum())

    return output


def lasso_wrapper_parallel(b_start, y, X, standardized=False, num_lambda=100, min_factor=0.001,
                           k_fold=False, num_k=5):

    # Question 1 Part F (a) + (b) + G

    """
        Function that performs the LASSO estimation through CDG + Active Set + SAFE for a pre-determined
         sequence of λ. Includes option for K-fold cross-validation.

        :param b_start: Initial guess for the parameter vector (may or may not include b0, which will be
         trimmed out if so)
        :param y: Outcome variable, vector of size N.
        :param X: Covariate variables (may or may not include ι), matrix of size N x P.
        :param standardized: Indicator for whether the data has been standardized.

        :param num_lambda: Number of λ values to include in the sequence.
        :param min_factor: The minimum value of λ as a fraction of λ_max

        :param warm_start: Option for using warm start initial conditions.

        As such the lambda sequence is given by:
        Sequence(λ) = log([λ_max * min_factor : num_lambda : λ_max])

        :param k_fold: Option to compute cross-validation results of lambda in a K-fold CV algorithm.
        :param num_k: The number of K-folds to use. Used only when k_fold is True.

        :return: List containing:
            - :estimate: Final coefficient vector estimates across different λ values,
            - :objectives: Vector containing LASSO objective across different λ values,
            - :lambda: string regarding which stopping criterion was used.
    """

    def parallel_cdg(lmbda_index):

        """
            :param lmbda_index: Index of the lambda vector corresponding to the lambda value
             used in parallelization of the LASSO algorithm.
        """

        lmbda = lmbda_vec[lmbda_index]

        # Running lasso_cdg for the given value of lambda. We use both the active_set and SAFE strategies.
        lasso_est = lasso_cdg(b_start, y, X, lmbda, eps=1e-6, max_iter=1000, standardized=standardized,
                              active_set=True, active_set_cycle=10, safe=True)

        return [lasso_est['estimate'][-1], lasso_est['objectives'][-1], lasso_est['status']]

    def parallel_k_fold(lmbda_index):
        """
            :param lmbda_index: Index of the lambda vector corresponding to the lambda value
             used in parallelization of the LASSO cross-validation algorithm.
        """

        lmbda, split_num, cv_error_k = lmbda_vec[lmbda_index], 0, np.zeros(num_k)
        kf = KFold(n_splits=num_k)
        iota = (X[:, 0] == X[:, 0].mean()).all()

        # Looping over each K-fold

        for train_index, test_index in kf.split(X):

            # Training to obtain LASSO estimate for the given lambda

            X_train, y_train = X[train_index, :], y[train_index]

            lasso_res_k = lasso_cdg(b_start=b_start, y=y_train, X=X_train, lmbda=lmbda,
                                    standardized=standardized,
                                    active_set=True, active_set_cycle=10, safe=True)

            lasso_est_k = lasso_res_k['estimate'][-1]

            # Using the test data to construct the prediction error

            X_test, y_test = X[test_index, :], y[test_index]

            cv_error_k[split_num] = np.mean(np.square(y_test - X_test @ lasso_est_k[~iota * 1:]))

            split_num = split_num + 1

        return [lmbda, cv_error_k.sum()]

    # Computing maximum lambda before LASSO estimate is exactly zero.
    lmbda_max = lambda_zero(y, X, standardized=standardized)



    # Computing sequence of lambdas over which LASSO is run based on the specification.
    lmbda_vec = np.flipud(np.linspace(start=min_factor, stop=1, num=num_lambda) * lmbda_max)

    # Number of cores used in parallelization.
    num_cores = multiprocessing.cpu_count()

    if k_fold is False:

        output = Parallel(n_jobs=num_cores)(delayed(parallel_cdg)(i) for i in range(num_lambda))

        return output

    else:

        output_CV = Parallel(n_jobs=num_cores)(delayed(parallel_k_fold)(i) for i in range(num_lambda))
        lmbda_CV = np.array(output_CV)[np.argmin(np.array(output_CV)[:, 1]), 0]

        lasso_CV = lasso_cdg(b_start=b_start, y=y, X=X, lmbda=lmbda_CV,
                                    standardized=standardized,
                                    active_set=True, active_set_cycle=10, safe=True)

        return output_CV, lmbda_CV, lasso_CV


## Question 2

In [3]:
def generateData_paper(N, p, sigmax): 
  # generate X
    X = random.multivariate_normal(size=N, mean=np.zeros(p), cov=(sigmax**2)*np.identity(p))
  # generate epsilon
    e = np.random.normal(size=N)
  # coefficient vector
    b = np.append(np.ones(5),np.zeros(p-5))
  # generate Y
    Y = X @ b + e

    dat=pd.concat([pd.DataFrame(Y,columns=['Y']),pd.DataFrame(X,columns=[i + j for i, j in zip(['X']*(p),map(str,list(range(1,p+1))))])],axis=1)

  # return dataset
    return(dat)


In [4]:
def rep_figure1_lasso(N,p,sigma_vec,S):
    l2 = np.zeros((3,sigma_vec.shape[0],S))
    cov_num = np.zeros((3,sigma_vec.shape[0],S))
    for k in range(S):
        for j in range(sigma_vec.shape[0]):
            data = generateData_paper(N, p, sigma_vec[j])
            lmbda = 2 * np.power((2.01*(sigma_vec[j]**2)*np.log(p)/N),1/2)
            lmbda_vec = np.array((lmbda, 2/3*lmbda, 1/3*lmbda ))
            for i in range(3):
                lasso = lasso_cdg(b_start=np.ones(p),y=data.Y, X=data[data.drop(['Y'], axis=1).columns].to_numpy(),lmbda=lmbda_vec[i],standardized=True)
                l2[i,j,k] = norm(lasso["estimate"][-1][1:]-np.append(np.ones(5),np.zeros(p-5)),ord=2)
                cov_num[i,j,k] =(np.nonzero(lasso["estimate"][-1])[0]).shape[0]-1
    return (l2,cov_num)

In [5]:
N = 500
p = 200
sigma_vec = np.array((1/2,1/3,1/4,1/5,1/6,1/7))
S=2
rs=rep_figure1_lasso(N,p,sigma_vec,S)
meanl2=np.mean(rs[0],axis=2)
meancov=np.mean(rs[1],axis=2)

In [21]:
def rep_figure2_lasso(N,p,sigma_vec,S):
    bias = np.zeros((3,sigma_vec.shape[0],S))
    cover = np.zeros((3,sigma_vec.shape[0],S))
    interval = np.zeros((3,sigma_vec.shape[0],S))
    for k in range(S):
        for j in range(sigma_vec.shape[0]):
            data = generateData_paper(N, p, sigma_vec[j])
            lmbda = 2 * np.power((2.01*(sigma_vec[j]**2)*np.log(p)/N),1/2)
            lmbda_vec = np.array((lmbda, 2/3*lmbda, 1/3*lmbda ))
            for i in range(3):
                lasso = doubleLasso_estimateCoefficients_fixedLambda(data, lmbda_vec[i], 0.05)
                bias[i,j,k] = lasso['estimate']
                cover[i,j,k] =(lasso['lb'] <= 1) * (1 <= lasso['ub'])
                interval[i,j,k] =  lasso['ub'] - lasso['lb'] 
    return (bias,cover,interval)


In [25]:
N = 500
p = 200
sigma_vec = np.array((1/2,1/3,1/4,1/5,1/6,1/7))
S=2
rs=rep_figure2_lasso(N,p,sigma_vec,S)
meanbias=np.mean(rs[0],axis=2)
meancover=np.mean(rs[1],axis=2)
meaninterval=np.mean(rs[1],axis=2)

IndexError: invalid index to scalar variable.

In [ ]:


print("Figure 1 for Lasso:")
fig, ax = plt.subplots(1, 2, figsize=(10, 5),  dpi=400, sharey='row')

label=[]
n=3
lmbda=np.array(('1','2/3','1/3'))
inv_sigma=np.array((2,3,4,5,6,7))
ax[0].set_prop_cycle('color',[plt.cm.coolwarm(i) for i in np.linspace(0, 1, n)])
for k in range(n):
    ax[0].plot(inv_sigma,meanl2[:,k],'--',marker="o",markersize=5)
    label=label+['${}\lambda$ '.format(lmbda[k])]
ax[0].legend(label,fontsize=8)
ax[0].set_xlabel(r"$1/\sigma_x$")
ax[0].set_ylabel(r"Estimation Error")


n=3
ax[1].set_prop_cycle('color',[plt.cm.coolwarm(i) for i in np.linspace(0, 1, n)])
for k in range(n):
    ax[1].plot(inv_sigma,meancov[:,k],'--',marker="o",markersize=5)
    label=label+['${}\lambda$ '.format(lmbda[k])]
ax[1].legend(label,fontsize=8)
ax[1].set_xlabel(r"$1/\sigma_x$")
ax[1].set_ylabel(r"Number of Selected Regressors")


In [ ]:
def generateData_paper2(N, p, sigmax): 
  # generate X
    X = random.multivariate_normal(size=N, mean=np.zeros(p), cov=(sigmax**2)*np.identity(p))
  # generate epsilon
    e = np.random.normal(size=N)
    v = np.random.normal(size=N)
  # coefficient vector
    b = np.append(np.ones(5),np.zeros(p-5))
    gamma = np.append(np.ones(5),np.zeros(p-5))
  # generate Y
    Y = X @ b + e
    D = X @ gamma + v

    dat=pd.concat([pd.DataFrame(Y,columns=['Y']),pd.DataFrame(D,columns=['D']),pd.DataFrame(X,columns=[i + j for i, j in zip(['X']*(p),map(str,list(range(1,p+1))))])],axis=1)

  # return dataset
    return(dat)


## Question 3

In [ ]:
def sim_generateData(N, rho): 
  # generate X
    X1 = np.random.normal(size=N)
    X2 = rho * X1 + np.sqrt(1-rho**2) * np.random.normal(size=N)
  
  # generate Y
    Y = X1 + (1/np.sqrt(N)) * X2 + np.random.normal(size=N)
    data = {'Y': Y, 'X1': X1, 'X2': X2}
    df = pd.DataFrame (data, columns = ['Y','X1','X2'],index=range(N))

  # return dataset
    return(df)


#### a)

In [ ]:
def pretest_estimateCoefficients(dat, alpha):
    # read dimensions
    N = dat.shape[0]
    dn= 1/(N**(1/4))
    X=dat.loc[:, dat.columns.isin(['X1','X2'])]
    mod = sm.OLS(dat.Y, X)
    res = mod.fit()  
    keyDict = {"coefficient","lb","ub","test"}
    output = dict([(key, []) for key in keyDict])
    if np.abs(res.params[1])>dn:
        output['coefficient']=res.params[0]
        output['lb']=res.conf_int(alpha=alpha, cols=None)[0][0]
        output['ub']=res.conf_int(alpha=alpha, cols=None)[1][0]
        output['test']= False
    else:
        X=dat.X1
        mod = sm.OLS(dat.Y, X)
        res = mod.fit()  
        output['coefficient']=res.params[0]
        output['lb']=res.conf_int(alpha=alpha, cols=None)[0][0]
        output['ub']=res.conf_int(alpha=alpha, cols=None)[1][0]
        output['test']= True    
    return output 

  



#### b)

In [ ]:
def pretest_simulateCoefficients(N, rho, alpha, S):
    results = pd.DataFrame(columns=("coefficient","lb","ub","test"))
    # perform Monte Carlo simulation
    for k in range(S): 
        dat = sim_generateData(N, rho)
        results=pd.concat([results,pd.DataFrame(pretest_estimateCoefficients(dat, alpha),index=[k])])

    return(results)

# set seed
random.seed(100)
# specify model
# list of N values
N_array = np.array((100, 200, 400, 700, 1000))
# correlation between X's
rho = 0.9
# significance level
alpha = 0.05
# number of Monte Carlo replications
S = 1000

result = pd.DataFrame(columns=("N","coverageProb","shortModelSelectionProb"))
k=0
# perform simulation
for i in N_array:

    # simulate pretest estimators
    results = pretest_simulateCoefficients(N=i, rho=rho, alpha=alpha, S=S)
    # check if each confidence interval contains the true value
    includeTrueValue = (results['lb'] <= 1) * (1 <= results['ub'])
    # return the result
    result=pd.concat([result,pd.DataFrame({"N":i,"coverageProb":includeTrueValue.mean(),"shortModelSelectionProb":results['test'].mean()},index=[k])])
    k=k+1
print(result)



### Question 4

In [27]:
def sim_generateData2(N, p, rho):
    # generate X
    X1 = np.random.normal(size=N)
    Xp = (rho / (p-1)) * np.transpose([X1]*(p-1))+ np.sqrt(0.5 * (1-rho**2) / (p-1)) * np.random.normal(size=(N,p-1))
    # generate Y
    # bind Y and X
    Y = X1 + ((1/np.sqrt(N)) * Xp).sum() + np.sqrt(0.5 * (1-rho**2)) * np.random.normal(size=N)
    dat=pd.concat([pd.DataFrame(Y,columns=['Y']),pd.DataFrame(X1,columns=['X1']),pd.DataFrame(Xp,columns=[i + j for i, j in zip(['X']*(p-1),map(str,list(range(2,p+1))))])],axis=1)
    
    return(dat)


#### a)

In [28]:
def doubleLasso_estimateCoefficients_fixedLambda(dat, lmbda, alpha):
  
    # read dimensions
    N = dat.shape[0]
    p = dat.shape[1] - 1
    keyDict = {"coefficient", "lb", "ub", "nreg"}
    output = dict([(key, []) for key in keyDict])
    
    #First Lasso
    lasso1 = lasso_cdg(b_start=np.ones(p-1),y=dat.Y, X=dat[dat.drop(['Y','X1'], axis=1).columns].to_numpy(),lmbda=lmbda[0])

    #List of covariates from the first lasso
    cov1 = dat[dat.drop(['Y'], axis=1).columns].columns[np.nonzero(lasso1["estimate"][-1])[0].tolist()]
    
    #Second Lasso
    lasso2 = lasso_cdg(b_start=np.ones(p-1),y=dat.X1, X=dat[dat.drop(['Y','X1'], axis=1).columns].to_numpy(),lmbda=lmbda[1])

    #List of covariates from the second lasso
    cov2 = dat[dat.drop(['Y'], axis=1).columns].columns[np.nonzero(lasso2["estimate"][-1])[0].tolist()]
    
    #Covariates from first or second lasso 
    X = dat.loc[:, dat.columns.isin(cov1) | dat.columns.isin(cov2)]
    #OLS
    mod = sm.OLS(dat.Y, X)
    res = mod.fit() 
    output['coefficient']=res.params[0]
    output['lb']=res.conf_int(alpha=alpha, cols=None)[0][0]
    output['ub']=res.conf_int(alpha=alpha, cols=None)[1][0]
    output['nreg']=res.params.shape[0]-1
    
    return output
    


#### b)

In [29]:
def doubleLasso_simulateCoefficients_fixedLambda(N, p, rho, lmbda, alpha, S):
  
    # prepare storage space
    results = pd.DataFrame(columns=("coefficient","lb","ub","nreg"))

  
    # perform Monte Carlo simulation
    for k in range(S):
        dat = sim_generateData2(N, p, rho)
        results=pd.concat([results,pd.DataFrame(doubleLasso_estimateCoefficients_fixedLambda(dat, lmbda, alpha),index=[k])])

  
    return(results)



# set seed
random.seed(100)

# specify model
# list of N values
N_array = np.array((100, 200, 400, 700, 1000))
# number of regressors
p = 3
# correlation between X's
rho = 0.9
# lasso penalty multipliers
lmbda = np.array((0.7, 0.7))
# significance level
alpha = 0.05
# number of Monte Carlo replications
S = 1000

result = pd.DataFrame(columns=("N","coverageProb","meanAdditionalRegressors"))
k=0
# perform simulation
for i in N_array:

    # simulate pretest estimators
    results = doubleLasso_simulateCoefficients_fixedLambda(N=i, p=p, rho=rho, lmbda=lmbda, alpha=alpha, S=S)
    # check if each confidence interval contains the true value
    includeTrueValue = (results['lb'] <= 1) * (1 <= results['ub'])
    # return the result
    result=pd.concat([result,pd.DataFrame({"N":i,"coverageProb":includeTrueValue.mean(),"meanAdditionalRegressors":results['nreg'].mean()},index=[k])])
    k=k+1
print(result)


KeyboardInterrupt: 

#### c)

In [30]:
def doubleLasso_estimateCoefficients_cvLambda(dat, alpha):
 
    # read dimensions
    N = dat.shape[0]
    p = dat.shape[1] - 1
    keyDict = {"coefficient", "lb", "ub", "nreg"}
    output = dict([(key, []) for key in keyDict])
    #First Lasso
    lasso1=lasso_wrapper_parallel(b_start=np.ones(p-1), y=dat.Y, X=dat[dat.drop(['Y','X1'], axis=1).columns].to_numpy(), k_fold=True, num_k=10)
    #List of covariates from the first lasso
    cov1 = dat[dat.drop(['Y'], axis=1).columns].columns[np.nonzero(lasso1[2]["estimate"][-1])[0].tolist()]
    
    #Second Lasso
    lasso2=lasso_wrapper_parallel(b_start=np.ones(p-1), y=dat.X1, X=dat[dat.drop(['Y','X1'], axis=1).columns].to_numpy(), k_fold=True, num_k=10)
    #List of covariates from the second lasso
    cov2 = dat[dat.drop(['Y'], axis=1).columns].columns[np.nonzero(lasso2[2]["estimate"][-1])[0].tolist()]
    
    #Covariates from first or second lasso 
    X = dat.loc[:, dat.columns.isin(cov1) | dat.columns.isin(cov2)]
    #OLS
    mod = sm.OLS(dat.Y, X)
    res = mod.fit() 
    output['coefficient']=res.params[0]
    output['lb']=res.conf_int(alpha=alpha, cols=None)[0][0]
    output['ub']=res.conf_int(alpha=alpha, cols=None)[1][0]
    output['nreg']=res.params.shape[0]-1
    
    return output
  


#### d)

In [31]:
def doubleLasso_simulateCoefficients_cvLambda(N, p, rho, alpha, S):
  
    # prepare storage space
    results = pd.DataFrame(columns=("coefficient","lb","ub","nreg"))

  
    # perform Monte Carlo simulation
    for k in range(S):
        dat = sim_generateData2(N, p, rho)
        results=pd.concat([results,pd.DataFrame(doubleLasso_estimateCoefficients_cvLambda(dat, alpha),index=[k])])

  
    return(results)



# set seed
random.seed(100)

# specify model
# list of N values
N_array = np.array((100, 200, 400, 700, 1000))
# number of regressors
p = 3
# correlation between X's
rho = 0.9
# significance level
alpha = 0.05
# number of Monte Carlo replications
S = 1000

result = pd.DataFrame(columns=("N","coverageProb","meanAdditionalRegressors"))
k=0
# perform simulation
for i in N_array:

    # simulate pretest estimators
    results = doubleLasso_simulateCoefficients_cvLambda(N=i, p=p, rho=rho, alpha=alpha, S=S)
    # check if each confidence interval contains the true value
    includeTrueValue = (results['lb'] <= 1) * (1 <= results['ub'])
    # return the result
    result=pd.concat([result,pd.DataFrame({"N":i,"coverageProb":includeTrueValue.mean(),"meanAdditionalRegressors":results['nreg'].mean()},index=[k])])
    k=k+1
print(result)


C:\Users\tugce\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:178: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "


      N  coverageProb  meanAdditionalRegressors
0   100           1.0                       2.0
1   200           0.8                       2.0
2   400           1.0                       2.0
3   700           1.0                       2.0
4  1000           0.9                       2.0
